In [257]:
# Sample Python code for user authorization
import os
import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import requests, re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")
import spacy
import en_core_web_sm
import en_core_web_lg
import en_vectors_web_lg
import en_core_web_md

from dateparser.search import search_dates
import arrow
import datetime
import dateutil.parser
import csv


In [ ]:
# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [ ]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

## Initialize Variables from Codebook

In [ ]:
# Get the channel title from pdf on the column "Channel Name"
channel_title='The Daily Wire'

In [ ]:
# Get the series title from pdf on the column "Series Name"
series = 'The Ben Shapiro Show'    

In [ ]:
# Input the filename 
csv_output = "benshapiro.csv"

## Get Channel Playlists

In [154]:
# channel_title='The Daily Wire'

In [165]:
# result = service.search().list(\
# q=channel_title,part='snippet',
# maxResults=5).execute()


In [308]:
def get_channel_id (channel_title='The Daily Wire'):
    result = service.search().list(
        q=channel_title,part='snippet',
        maxResults=5).execute()
    for res in result['items']:
        if(res['snippet']['title']==channel_title):
            print(res['id']['channelId'])
            channel_id = res['id']['channelId'] 
            break
    return channel_id
    

In [ ]:
def transform_date(input_date_str,new_timezone=new_timezone):
    video_date_posted_iso = dateutil.parser.parse(input_date_str).astimezone(new_timezone)
    video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
    return video_date_posted,video_date_posted_iso
def transform_date_custom(input_date_str,new_timezone=new_timezone):
    video_date_posted_iso = dateutil.parser.parse(input_date_str).astimezone(new_timezone)
    video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
    return video_date_posted
def transform_date_iso(input_date_str,new_timezone=new_timezone):
    video_date_posted_iso = dateutil.parser.parse(input_date_str).astimezone(new_timezone)
    return video_date_posted_iso

In [365]:
def set_flags (flag):
    video=0
    comment=0
    reply=0
    if flag=='video':
        video=1
    elif flag=='comment':
        comment=1
    elif flag=='reply':
        reply=1
    return video,comment,reply

In [176]:
# Search the channelId using the channel title
channel_id = get_channel_id(channel_title=channel_title)

UCaeO5vkdj5xOQHp4UmIN6dw


## Get Playlists from Channel

In [191]:
# series = 'The Ben Shapiro Show'                        

In [315]:
nextPageToken=''
while True:
    playlist_result = service.playlists().list(\
    channelId=channel_id,part='snippet,contentDetails',\
    maxResults=50,pageToken=nextPageToken).execute()

    #This will check if the given playlist is in youtube playlist

    for playlist in playlist_result['items']:
        if playlist['snippet']['title']==series:
            print(series)
            print(playlist['id'])
            playlist_id = playlist['id']
            video_count = playlist['contentDetails']['itemCount']
            break
    try:
        nextPageToken = playlist_result['nextPageToken']
    except:
        break

The Ben Shapiro Show
PLlVjFVMet3OXfEi7b2w50OUc7QMnpfKqK


In [316]:
video_count

585

## Get videos in playlist

In [223]:
result_vids = service.playlistItems().list(\
playlistId=playlist_id,part='snippet,contentDetails',\
maxResults=max_results).execute()
                                           

### Write video details to CSV

In [300]:
video =1
comment =0
reply =0
comment_op=''
reply_op=''
transcript = ''
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")       
cutoff_date = datetime.datetime(2018, 12, 2, 0, 0, 0, tzinfo=new_timezone)
nextPageToken=''
max_results=50
#  delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['index', 'video_id','video_url',
                 'video_title',"channel",'series','video_snippet',
                 'video','comment','reply','video_op',
                 'comment_op','reply_op','date_posted',
                 'content','date_scrape']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for i in range(int(video_count/max_results) +1):
        result_vids = service.playlistItems().list(
            playlistId=playlist_id,part='snippet,contentDetails',
            maxResults=max_results,pageToken=nextPageToken).execute()
        for result_item in result_vids['items'] :
            video_date_posted = result_item['contentDetails']['videoPublishedAt']
            video_date_posted_iso = dateutil.parser.parse(video_date_posted).astimezone(new_timezone)
            if(cutoff_date<video_date_posted_iso):
                continue
            video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
            video_id = result_item['contentDetails']['videoId']
            #     video_index = 
            video_index = result_item['snippet']['position']
            video_title = result_item['snippet']['title']
            video_snippet = result_item['snippet']['description']    
            video_op =result_item['snippet']['channelTitle']
            my_url='https://www.youtube.com/watch?v=%s' %(video_id)
#             vid_idx=i*max_results + int(video_index)
            display(video_title,vid_idx)

            x =  { "index": video_index,"video_id": video_id, "video_url":my_url,
                "video_title":video_title, "channel":channel_title,
                "series":series, "video_snippet":video_snippet,
                "video":video,"comment":comment,"reply":reply,
                "video_op":video_op, "comment_op":comment_op,
                "reply_op":reply_op,"date_posted":video_date_posted,
                "content":transcript,"date_scrape":date_scrape}   
            writer.writerow(x)
            try:
                nextPageToken = result_vids['nextPageToken']
            
            except:
                "End of list"


        

'Russian Roulette | The Ben Shapiro Show Ep. 670'

1100

'The ‘Real’ Jews | The Ben Shapiro Show Ep. 669'

1100

'Mulling Over Mueller | The Ben Shapiro Show Ep. 668'

...

'The Ben Shapiro Show Ep. 112 - There Are Things Worth Fighting For'

1100

'The Ben Shapiro Show Ep. 110 - Can The Trump Train Turn Hillary Into Roadkill?'

1100

KeyError: 'videoPublishedAt'

In [292]:
video =1
comment =0
reply =0
comment_op=''
reply_op=''
transcript = ''
cutoff_date = datetime.datetime(2018, 12, 2, 0, 0, 0, tzinfo=new_timezone)
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")       
result_vids = service.playlistItems().list(
    playlistId=playlist_id,part='snippet,contentDetails',
    maxResults=1).execute()
for result_item in result_vids['items'] :
    video_id = result_item['contentDetails']['videoId']
    #     video_index = 
    video_index = result_item['snippet']['position']
    video_title = result_item['snippet']['title']
    video_snippet = result_item['snippet']['description']    
    video_op =result_item['snippet']['channelTitle']
    video_date_posted = result_item['contentDetails']['videoPublishedAt']
    video_date_posted_iso = dateutil.parser.parse(video_date_posted).astimezone(new_timezone)
    video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
    display(video_date_posted)
    display(video_date_posted_iso<cutoff_date)
    my_url='https://www.youtube.com/watch?v=%s' %(video_id)
# nextPageToken = result_vids['nextPageToken']
    x =  { "video_id": video_id, "video_url":my_url,
        "video_title":video_title, "channel":channel_title,
        "series":series, "video_snippet":video_snippet,
        "video":video,"comment":comment,"reply":reply,
        "video_op":video_op, "comment_op":comment_op,
        "reply_op":reply_op,"date_posted":video_date_posted,
        "content":transcript,"date_scrape":date_scrape}        

'2018_12_10_15_07_09'

False

# Parse a single Video

In [11]:
video_id = 'sK7euOOQK-4'
my_url='https://www.youtube.com/watch?v=%s' %(video_id)

In [14]:
individual_vid = service.videos().list(\
id='sK7euOOQK-4',part='snippet,contentDetails').execute()
                                           

In [15]:
individual_vid

{'kind': 'youtube#videoListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/iTZaw3UujcGDtlci2Y2JsYTt82M"',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1},
 'items': [{'kind': 'youtube#video',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/2DQo9D-gAQH3P_sbRk6K6ltZXSo"',
   'id': 'sK7euOOQK-4',
   'snippet': {'publishedAt': '2018-12-09T13:00:05.000Z',
    'channelId': 'UCnQC_G5Xsjhp9fEJKuIcrSw',
    'title': 'Sunday Special Ep 30: David Limbaugh',
    'description': 'Author and columnist David Limbaugh joins Ben to discuss how Christianity and Judaism have shaped America, how his thinking on Donald Trump has changed since 2016, and whether 2020 will be better than 2018 for Republicans.\n\nSubscribe to the Daily Wire to watch the bonus question! https://bit.ly/2q0wopL\n\nCheck Ben out on:\nDaily Wire website: www.dailywire.com\nFacebook: https://www.facebook.com/officialbenshapiro\nTwitter: https://twitter.com/benshapiro\nInstagram: @officialbenshapiro\n \nCheck David Limbaugh out on:\n

In [23]:
video_title =individual_vid['items'][0]['snippet']['title']
video_title

'Sunday Special Ep 30: David Limbaugh'

In [31]:
video =1
comment=0
reply=0

In [27]:
video_date_posted =individual_vid['items'][0]['snippet']['publishedAt']
video_date_posted

'2018-12-09T13:00:05.000Z'

In [ ]:
video_date_posted =individual_vid['items'][0]['snippet']['publishedAt']
video_date_posted

In [47]:
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")

In [46]:
video_op =individual_vid['items'][0]['snippet']['channelTitle']
comment_op=''
reply_op=''
video_op

'Ben Shapiro'

In [45]:
video_snippet = individual_vid['items'][0]['snippet']['description']
video_snippet

'Author and columnist David Limbaugh joins Ben to discuss how Christianity and Judaism have shaped America, how his thinking on Donald Trump has changed since 2016, and whether 2020 will be better than 2018 for Republicans.\n\nSubscribe to the Daily Wire to watch the bonus question! https://bit.ly/2q0wopL\n\nCheck Ben out on:\nDaily Wire website: www.dailywire.com\nFacebook: https://www.facebook.com/officialbenshapiro\nTwitter: https://twitter.com/benshapiro\nInstagram: @officialbenshapiro\n \nCheck David Limbaugh out on:\nTwitter: https://twitter.com/davidlimbaugh'

In [51]:
x =  { "video_id": video_id, "video_url":my_url,\
      "video_title":video_title, \
#       "channel":channel,\
#      "series":series, \
     "video_snippet":video_snippet,\
     "video":video,"comment":comment,"reply":reply,\
     "video_op":video_op, "comment_op":comment_op,\
     "reply_op":reply_op,"date_posted":video_date_posted,\
#      "content":transcript,\
      "date_scrape":date_scrape}

In [52]:
x

{'video_id': 'sK7euOOQK-4',
 'video_url': 'https://www.youtube.com/watch?v=sK7euOOQK-4',
 'video_title': 'Sunday Special Ep 30: David Limbaugh',
 'video_snippet': 'Author and columnist David Limbaugh joins Ben to discuss how Christianity and Judaism have shaped America, how his thinking on Donald Trump has changed since 2016, and whether 2020 will be better than 2018 for Republicans.\n\nSubscribe to the Daily Wire to watch the bonus question! https://bit.ly/2q0wopL\n\nCheck Ben out on:\nDaily Wire website: www.dailywire.com\nFacebook: https://www.facebook.com/officialbenshapiro\nTwitter: https://twitter.com/benshapiro\nInstagram: @officialbenshapiro\n \nCheck David Limbaugh out on:\nTwitter: https://twitter.com/davidlimbaugh',
 'video': 1,
 'comment': 0,
 'reply': 0,
 'video_op': 'Ben Shapiro',
 'comment_op': '',
 'reply_op': '',
 'date_posted': '2018-12-09T13:00:05.000Z',
 'date_scrape': '2018_12_10_07_14_17'}

## Extract Comments

In [317]:
video_id

'UfxIf3frJuE'

In [304]:
result_comments= service.commentThreads().list(
    part="snippet,replies",
    videoId=video_id,
    textFormat="plainText",
    maxResults=2
  ).execute()

In [ ]:
result_comments['pageInfo']['totalResults']

In [307]:
for comment_ind in result_comments['items']:
    display(comment_ind['snippet']['topLevelComment']['snippet']['authorDisplayName'])
    display(comment_ind['snippet']['topLevelComment']['snippet']['publishedAt'])

'BlackGreninja1'

'2018-12-02T23:43:20.000Z'

'Cj Schneidt'

'2018-11-03T01:00:32.000Z'

In [55]:
result_comments

{'kind': 'youtube#commentThreadListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/R6US-T41ng5wGiw4-z4SvQo906U"',
 'nextPageToken': 'QURTSl9pMUJSNGI1clJmNVhlNEh4RFl3TlQ3T3RnVkVLaGlrd3hpcDI5TDQ3X2JfWF9Eemg2cnlkWmpkSEtaaDdGa05ENTdrZDJqaktic1dLRmVOd0NrT1RyN19xTEtLb0JORXF5UHN5Nk16bzVybGJTWUthMjR0bHROWU5XTlF1SXM=',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1},
 'items': [{'kind': 'youtube#commentThread',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/j2wr-NGhrxTtlJV8g8w6EuJCKWQ"',
   'id': 'Ugxs4x23BQyM4_bKaWh4AaABAg',
   'snippet': {'videoId': 'sK7euOOQK-4',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/K_ZyBbCbe4Z5ER-UgE4XM0DtKSA"',
     'id': 'Ugxs4x23BQyM4_bKaWh4AaABAg',
     'snippet': {'authorDisplayName': 'DomManInT1',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/-UeLVQy7FA-o/AAAAAAAAAAI/AAAAAAAAAAA/Ylc24g47Z3M/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
      'authorChannelUrl': 'http://www.youtube.com/channel/UCXVjqsEe5M-W7Rh

In [ ]:
result_comments

In [57]:
video =0
comment=1
reply=0

In [70]:
video_op =video_op
comment_op=result_comments['items'][0]['snippet']['topLevelComment']['snippet']['authorDisplayName']
reply_op=''
display(video_op)
display(comment_op)

'Ben Shapiro'

'DomManInT1'

In [82]:
comment_date_posted =result_comments['items'][0]['snippet']['topLevelComment']['snippet']['publishedAt']

In [89]:
comment_content = result_comments['items'][0]['snippet']['topLevelComment']['snippet']['textOriginal']
comment_content

'A lot of chattering, stuttering, and broken sentences from the guest. Not very persuasive.'

In [90]:
x =  { "video_id": video_id, "video_url":my_url,\
      "video_title":video_title, \
#       "channel":channel,\
#      "series":series, \
     "video_snippet":video_snippet,\
     "video":video,"comment":comment,"reply":reply,\
     "video_op":video_op, "comment_op":comment_op,\
     "reply_op":reply_op,"date_posted":comment_date_posted,\
     "content":comment_content,\
      "date_scrape":date_scrape}
x

{'video_id': 'sK7euOOQK-4',
 'video_url': 'https://www.youtube.com/watch?v=sK7euOOQK-4',
 'video_title': 'Sunday Special Ep 30: David Limbaugh',
 'video_snippet': 'Author and columnist David Limbaugh joins Ben to discuss how Christianity and Judaism have shaped America, how his thinking on Donald Trump has changed since 2016, and whether 2020 will be better than 2018 for Republicans.\n\nSubscribe to the Daily Wire to watch the bonus question! https://bit.ly/2q0wopL\n\nCheck Ben out on:\nDaily Wire website: www.dailywire.com\nFacebook: https://www.facebook.com/officialbenshapiro\nTwitter: https://twitter.com/benshapiro\nInstagram: @officialbenshapiro\n \nCheck David Limbaugh out on:\nTwitter: https://twitter.com/davidlimbaugh',
 'video': 0,
 'comment': 1,
 'reply': 0,
 'video_op': 'Ben Shapiro',
 'comment_op': 'DomManInT1',
 'reply_op': '',
 'date_posted': '2018-12-10T06:12:31.000Z',
 'content': 'A lot of chattering, stuttering, and broken sentences from the guest. Not very persuasive.'

In [323]:
result_comments= service.commentThreads().list(
    part="snippet,replies",
    videoId=video_id,
    textFormat="plainText",
    maxResults=100
  ).execute()

In [369]:
video =0
comment =1
reply =0
comment_op=''
reply_op=''
transcript = ''
# cutoff_date = datetime.datetime(2018, 12, 2, 0, 0, 0, tzinfo=new_timezone)
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")       
# for result_item in result_vids['items'] :
#     video_id = result_item['contentDetails']['videoId']
#     #     video_index = 
#     video_index = result_item['snippet']['position']
#     video_title = result_item['snippet']['title']
#     video_snippet = result_item['snippet']['description']    
#     video_op =result_item['snippet']['channelTitle']
#     video_date_posted = result_item['contentDetails']['videoPublishedAt']
#     video_date_posted_iso = dateutil.parser.parse(video_date_posted).astimezone(new_timezone)
#     video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
#     display(video_date_posted)
#     display(video_date_posted_iso<cutoff_date)
#     my_url='https://www.youtube.com/watch?v=%s' %(video_id)
# # nextPageToken = result_vids['nextPageToken']
#     x =  { "video_id": video_id, "video_url":my_url,
#         "video_title":video_title, "channel":channel_title,
#         "series":series, "video_snippet":video_snippet,
#         "video":video,"comment":comment,"reply":reply,
#         "video_op":video_op, "comment_op":comment_op,
#         "reply_op":reply_op,"date_posted":video_date_posted,
#         "content":transcript,"date_scrape":date_scrape}   
    
# result_comments
for comment_ind in result_comments['items']:
    reply_op=''
    comment_op = comment_ind['snippet']['topLevelComment']['snippet']['authorDisplayName']
    display(comment_op)
    comment_date_posted = comment_ind['snippet']['topLevelComment']['snippet']['publishedAt']
#     display(comment_ind['snippet']['topLevelComment']['snippet']['publishedAt'])
    comment_index = result_item['snippet']['position']
    comment_content = comment_ind['snippet']['topLevelComment']['snippet']['textOriginal']
    display(comment_content)
    display(comment_index)
    video,comment,reply=set_flags(flag='comment')
    x =  { "video_id": video_id, 
        "video":video,"comment":comment,"reply":reply,
        "comment_op":comment_op,
        "reply_op":reply_op,"date_posted":comment_date_posted,
        "content":comment_content,"date_scrape":date_scrape}  
    display(x)    
    
    break
# for x in result_comments['items']:
#     if (x['snippet']['totalReplyCount']>1):
# #     'dfdf'.
# #         print(x)
#         display(x)
#         test=x
#         break

'BlackGreninja1'

'Who else came here to find the earliest episode???'

583

{'video_id': 'UfxIf3frJuE',
 'video': 0,
 'comment': 1,
 'reply': 0,
 'comment_op': 'BlackGreninja1',
 'reply_op': '',
 'date_posted': '2018-12-02T23:43:20.000Z',
 'content': 'Who else came here to find the earliest episode???',
 'date_scrape': '2018_12_11_04_33_38'}

In [379]:
for comment_ind in result_comments['items']:
    # display(comment_ind)
    if comment_ind['snippet']['totalReplyCount']>1:
        display('True')
        display(comment_ind['snippet']['totalReplyCount'])
        for reply_result in comment_ind['replies']['comments']:
            reply_op = reply_result['snippet']['authorDisplayName']
            reply_content = reply_result['snippet']['textOriginal']
            reply_date_posted = transform_date_custom(reply_result['snippet']['publishedAt'])
            video,comment,reply=set_flags(flag='reply')
            x =  { "video_id": video_id, 
                "video":video,"comment":comment,"reply":reply,
                "comment_op":comment_op,
                "reply_op":reply_op,"date_posted":reply_date_posted,
                "content":reply_content,"date_scrape":date_scrape}  
            display(x)
    # display()
    # display(len(comment_ind['replies']))

'True'

2

{'video_id': 'UfxIf3frJuE',
 'video': 0,
 'comment': 0,
 'reply': 1,
 'comment_op': 'BlackGreninja1',
 'reply_op': 'Cj Schneidt',
 'date_posted': '2018_11_02_21_05_22',
 'content': 'Hes done a near 180 on trump',
 'date_scrape': '2018_12_11_04_36_27'}

{'video_id': 'UfxIf3frJuE',
 'video': 0,
 'comment': 0,
 'reply': 1,
 'comment_op': 'BlackGreninja1',
 'reply_op': 'Cj Schneidt',
 'date_posted': '2018_11_02_21_05_09',
 'content': 'Right here',
 'date_scrape': '2018_12_11_04_36_27'}

'True'

...

{'video_id': 'UfxIf3frJuE',
 'video': 0,
 'comment': 0,
 'reply': 1,
 'comment_op': 'BlackGreninja1',
 'reply_op': 'Famas',
 'date_posted': '2016_05_03_16_04_31',
 'content': '+FayceTwitch cuck',
 'date_scrape': '2018_12_11_04_36_27'}

'True'

2

{'video_id': 'UfxIf3frJuE',
 'video': 0,
 'comment': 0,
 'reply': 1,
 'comment_op': 'BlackGreninja1',
 'reply_op': 'Ryan Rebischke',
 'date_posted': '2016_05_23_17_25_48',
 'content': "+cazinger I completely agree!  i couldn't figure out where he went!",
 'date_scrape': '2018_12_11_04_36_27'}

{'video_id': 'UfxIf3frJuE',
 'video': 0,
 'comment': 0,
 'reply': 1,
 'comment_op': 'BlackGreninja1',
 'reply_op': 'cazinger',
 'date_posted': '2016_05_10_20_30_24',
 'content': '+Alex McBrayer \nI would just ask that next time he switches from posting these videos under one channel to another channel, it would be nice if he posted something under the old channel telling people where to find his new videos.',
 'date_scrape': '2018_12_11_04_36_27'}

In [384]:
video_id
my_url
video_title

'The Ben Shapiro Show Ep. 110 - Can The Trump Train Turn Hillary Into Roadkill?'

In [387]:
comment_op=''
reply_op=''
transcript = ''
# cutoff_date = datetime.datetime(2018, 12, 2, 0, 0, 0, tzinfo=new_timezone)
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")       
nextPageToken=''
csv_output='test_1vid_comment.csv'
with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['index', 'video_id','video_url',
                 'video_title',"channel",'series','video_snippet',
                 'video','comment','reply','video_op',
                 'comment_op','reply_op','date_posted',
                 'content','date_scrape']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()    
    while True:
        result_comments= service.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100,
            pageToken = nextPageToken
          ).execute()

        for comment_ind in result_comments['items']:
            reply_op=''
            comment_op = comment_ind['snippet']['topLevelComment']['snippet']['authorDisplayName']
            display(comment_op)
            comment_date_posted = comment_ind['snippet']['topLevelComment']['snippet']['publishedAt']
        #     display(comment_ind['snippet']['topLevelComment']['snippet']['publishedAt'])
            comment_index = comment_ind['snippet']['topLevelComment']['id']
            comment_content = comment_ind['snippet']['topLevelComment']['snippet']['textOriginal']
            video,comment,reply=set_flags(flag='comment')
    #         x =  { "video_id": video_id, 
    #             "video":video,"comment":comment,"reply":reply,
    #             "comment_op":comment_op,
    #             "reply_op":reply_op,"date_posted":comment_date_posted,
    #             "content":comment_content,"date_scrape":date_scrape}  
            x =  { "index": comment_index,"video_id": video_id, "video_url":my_url,
                "video_title":video_title, "channel":channel_title,
                "series":series, "video_snippet":video_snippet,
                "video":video,"comment":comment,"reply":reply,
                "video_op":video_op,"comment_op":comment_op,
                "reply_op":reply_op,"date_posted":comment_date_posted,
                "content":comment_content,"date_scrape":date_scrape}          

            writer.writerow(x)        
    #         display(x)  
            print(str(comment_ind['snippet']['totalReplyCount']) + ' replies')

            if comment_ind['snippet']['totalReplyCount']>0:
                display('True')
                display(comment_ind['snippet']['totalReplyCount'])
                for reply_result in comment_ind['replies']['comments']:
                    reply_op = reply_result['snippet']['authorDisplayName']
                    reply_content = reply_result['snippet']['textOriginal']
                    reply_date_posted = transform_date_custom(reply_result['snippet']['publishedAt'])
                    video,comment,reply=set_flags(flag='reply')
    #                 x =  { "video_id": video_id, 
    #                     "video":video,"comment":comment,"reply":reply,
    #                     "comment_op":comment_op,
    #                     "reply_op":reply_op,"date_posted":reply_date_posted,
    #                     "content":reply_content,"date_scrape":date_scrape}  
    #                 display(x)   
                    x =  {"index": comment_index,"video_id": video_id, "video_url":my_url,
                        "video_title":video_title, "channel":channel_title,
                        "series":series, "video_snippet":video_snippet,
                        "video":video,"comment":comment,"reply":reply,
                        "video_op":video_op, "comment_op":comment_op,
                        "reply_op":reply_op,"date_posted":reply_date_posted,
                        "content":reply_content,"date_scrape":date_scrape} 
                    writer.writerow(x)        

        try:
            nextPageToken = result_comments['nextPageToken']
        except:
            break
            "End of list"

'BlackGreninja1'

1 replies


'True'

1

'Cj Schneidt'

...

3

'Alex McBrayer'

2 replies


'True'

2

'UgjekX7Z104szHgCoAEC.8DQib-coLX48DiJC_8-WEc'

## End-to-end

In [ ]:
video =1
comment =0
reply =0
comment_op=''
reply_op=''
transcript = ''
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")       
cutoff_date = datetime.datetime(2018, 12, 2, 0, 0, 0, tzinfo=new_timezone)
nextPageToken=''
max_results=50
#  delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['index', 'video_id','video_url',
                 'video_title',"channel",'series','video_snippet',
                 'video','comment','reply','video_op',
                 'comment_op','reply_op','date_posted',
                 'content','date_scrape']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for i in range(int(video_count/max_results) +1):
        result_vids = service.playlistItems().list(
            playlistId=playlist_id,part='snippet,contentDetails',
            maxResults=max_results,pageToken=nextPageToken).execute()
        for result_item in result_vids['items'] :
            video_date_posted = result_item['contentDetails']['videoPublishedAt']
            video_date_posted_iso = dateutil.parser.parse(video_date_posted).astimezone(new_timezone)
            if(cutoff_date<video_date_posted_iso):
                continue
            video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
            video_id = result_item['contentDetails']['videoId']
            #     video_index = 
            video_index = result_item['snippet']['position']
            video_title = result_item['snippet']['title']
            video_snippet = result_item['snippet']['description']    
            video_op =result_item['snippet']['channelTitle']
            my_url='https://www.youtube.com/watch?v=%s' %(video_id)
#             vid_idx=i*max_results + int(video_index)
            display(video_title,vid_idx)

            x =  { "index": video_index,"video_id": video_id, "video_url":my_url,
                "video_title":video_title, "channel":channel_title,
                "series":series, "video_snippet":video_snippet,
                "video":video,"comment":comment,"reply":reply,
                "video_op":video_op, "comment_op":comment_op,
                "reply_op":reply_op,"date_posted":video_date_posted,
                "content":transcript,"date_scrape":date_scrape}   
            writer.writerow(x)
            try:
                nextPageToken = result_vids['nextPageToken']
            
            except:
                "End of list"


## Extract Reply

In [ ]:
# result_comments['items'][0]['snippet']['totalReplyCount']

In [138]:
for x in result_comments['items']:
    if (x['snippet']['totalReplyCount']>1):
#     'dfdf'.
#         print(x)
        display(x)
        test=x
        break
# result_comments['items'][0]['id']

{'kind': 'youtube#commentThread',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/FH1ujImpbefhkzjc2bVMiZfpabE"',
 'id': 'Ugwr8aTKL2Ac4B7szNV4AaABAg',
 'snippet': {'videoId': 'sK7euOOQK-4',
  'topLevelComment': {'kind': 'youtube#comment',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/7zz--rViwwzKXfzl-auWtiMMTf8"',
   'id': 'Ugwr8aTKL2Ac4B7szNV4AaABAg',
   'snippet': {'authorDisplayName': 'Devorah BatMiriamGoldaVMordecai Bayer',
    'authorProfileImageUrl': 'https://yt3.ggpht.com/-eHEQ1TjpZ60/AAAAAAAAAAI/AAAAAAAAAAA/QmMPDTLj7ZA/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
    'authorChannelUrl': 'http://www.youtube.com/channel/UCpGCxjLA9RBVi1vGT9vC2KQ',
    'authorChannelId': {'value': 'UCpGCxjLA9RBVi1vGT9vC2KQ'},
    'videoId': 'sK7euOOQK-4',
    'textDisplay': 'Ben, sorry, I as a Torah observant Jew, can’t hold by followers of Yesha.',
    'textOriginal': 'Ben, sorry, I as a Torah observant Jew, can’t hold by followers of Yesha.',
    'canRate': True,
    'viewerRating': 'none',
    'likeCount': 2,
    'p

In [139]:
reply_op = x['replies']['comments'][0]['snippet']['authorDisplayName']
reply_op

'Jeh Jeh'

In [140]:
reply_content = x['replies']['comments'][0]['snippet']['textOriginal']
reply_content

"Why? Yeshua was a Jew and went to temple. He never put down the Jews and came to take their sins upon him. I'm curious as to why the Jews stopped sacrificing live animals."

In [141]:
reply_date_posted = x['replies']['comments'][0]['snippet']['publishedAt']
reply_date_posted

'2018-12-10T12:17:49.000Z'

In [142]:
video =0
comment=0
reply=1

In [143]:
x =  { "video_id": video_id, "video_url":my_url,\
      "video_title":video_title, \
#       "channel":channel,\
#      "series":series, \
     "video_snippet":video_snippet,\
     "video":video,"comment":comment,"reply":reply,\
     "video_op":video_op, "comment_op":comment_op,\
     "reply_op":reply_op,"date_posted":reply_date_posted,\
     "content":reply_content,\
      "date_scrape":date_scrape}
x

{'video_id': 'sK7euOOQK-4',
 'video_url': 'https://www.youtube.com/watch?v=sK7euOOQK-4',
 'video_title': 'Sunday Special Ep 30: David Limbaugh',
 'video_snippet': 'Author and columnist David Limbaugh joins Ben to discuss how Christianity and Judaism have shaped America, how his thinking on Donald Trump has changed since 2016, and whether 2020 will be better than 2018 for Republicans.\n\nSubscribe to the Daily Wire to watch the bonus question! https://bit.ly/2q0wopL\n\nCheck Ben out on:\nDaily Wire website: www.dailywire.com\nFacebook: https://www.facebook.com/officialbenshapiro\nTwitter: https://twitter.com/benshapiro\nInstagram: @officialbenshapiro\n \nCheck David Limbaugh out on:\nTwitter: https://twitter.com/davidlimbaugh',
 'video': 0,
 'comment': 0,
 'reply': 1,
 'video_op': 'Ben Shapiro',
 'comment_op': 'DomManInT1',
 'reply_op': 'Jeh Jeh',
 'date_posted': '2018-12-10T12:17:49.000Z',
 'content': '2018-12-10T12:17:49.000Z',
 'date_scrape': '2018_12_10_07_14_17'}

In [166]:
display(test['replies']['comments'][0]['snippet'])
#author
display(test['replies']['comments'][0]['snippet']['authorDisplayName'])
#text
display(test['replies']['comments'][0]['snippet']['textOriginal'])
#replies to

display(test['snippet']['topLevelComment']['snippet']['authorDisplayName'])

{'authorDisplayName': 'Firenze 64',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/-KjHNkwHdaoY/AAAAAAAAAAI/AAAAAAAAAAA/N09yThjvOYg/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
 'authorChannelUrl': 'http://www.youtube.com/channel/UCQJLqOsy7dx7cMVHA-vozfA',
 'authorChannelId': {'value': 'UCQJLqOsy7dx7cMVHA-vozfA'},
 'videoId': 'A1tqTQkC36A',
 'textDisplay': 'They weren’t hooray',
 'textOriginal': 'They weren’t hooray',
 'parentId': 'UgwQTQvknQyoCTRAkUp4AaABAg',
 'canRate': True,
 'viewerRating': 'none',
 'likeCount': 0,
 'publishedAt': '2018-11-07T10:50:15.000Z',
 'updatedAt': '2018-11-07T10:50:15.000Z'}

'Firenze 64'

'They weren’t hooray'

'jdski 892'

'jdski 892'

In [131]:
test = 'dfdf'

In [ ]:
test.s

In [283]:
result_comments_few= service.commentThreads().list(
    part="snippet,replies",
    videoId='mJxcNiyWd8Q',
    textFormat="plainText",
    maxResults=2
  ).execute()

In [284]:
result_comments_few

{'kind': 'youtube#commentThreadListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/iOw6nirNiAyVoYcvAedB3INpQS8"',
 'pageInfo': {'totalResults': 2, 'resultsPerPage': 2},
 'items': [{'kind': 'youtube#commentThread',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/VrmbgIMKDIbsCF73fKshmMornZ8"',
   'id': 'UgwMdxC3HCxeIh70Vmd4AaABAg',
   'snippet': {'videoId': 'mJxcNiyWd8Q',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/KvRJpKp6V0ULnY3TAbU7Avkn5Ag"',
     'id': 'UgwMdxC3HCxeIh70Vmd4AaABAg',
     'snippet': {'authorDisplayName': 'Maribelle Cual',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/-P3AC-PdHzw8/AAAAAAAAAAI/AAAAAAAAAAA/EbJl_ea-Mmo/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
      'authorChannelUrl': 'http://www.youtube.com/channel/UCnJrRJehfcW4vSJMI3MtJAA',
      'authorChannelId': {'value': 'UCnJrRJehfcW4vSJMI3MtJAA'},
      'videoId': 'mJxcNiyWd8Q',
      'textDisplay': 'U dont know how to hurt me😭😭',
      'textOriginal': 'U dont know

In [77]:
result_comments['items'][0]['snippet']

{'videoId': 'sK7euOOQK-4',
 'topLevelComment': {'kind': 'youtube#comment',
  'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/3ZtV8-2TTVaaXlxn8pHyzAYSkpA"',
  'id': 'UgyO1EYSuRR3VoBekHd4AaABAg',
  'snippet': {'authorDisplayName': 'The Poser Podcast',
   'authorProfileImageUrl': 'https://yt3.ggpht.com/-XksVdHtfb_k/AAAAAAAAAAI/AAAAAAAAAAA/58FRXDO6NMg/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
   'authorChannelUrl': 'http://www.youtube.com/channel/UCNoq4F8Z66onlLtMB6ilCKA',
   'authorChannelId': {'value': 'UCNoq4F8Z66onlLtMB6ilCKA'},
   'videoId': 'sK7euOOQK-4',
   'textDisplay': 'This guy is piss weak',
   'textOriginal': 'This guy is piss weak',
   'canRate': True,
   'viewerRating': 'none',
   'likeCount': 0,
   'publishedAt': '2018-12-10T05:50:01.000Z',
   'updatedAt': '2018-12-10T05:50:01.000Z'}},
 'canReply': True,
 'totalReplyCount': 0,
 'isPublic': True}

In [ ]:
result

In [113]:
result_com = service.comments().list(
  part="snippet",
  parentId='UgwQTQvknQyoCTRAkUp4AaABAg',
  textFormat="plainText"
).execute()

In [110]:
result_com

{'kind': 'youtube#commentListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/6nUGxaj7vAVtBKWheQdcDliTN-0"',
 'pageInfo': {'resultsPerPage': 20},
 'items': [{'kind': 'youtube#comment',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/29SGnIYALXlm7Z0TrbsHeFx-jzQ"',
   'id': 'UgwQTQvknQyoCTRAkUp4AaABAg.8nK74henT5a8nLaWiger9N',
   'snippet': {'authorDisplayName': 'Firenze 64',
    'authorProfileImageUrl': 'https://yt3.ggpht.com/-KjHNkwHdaoY/AAAAAAAAAAI/AAAAAAAAAAA/N09yThjvOYg/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
    'authorChannelUrl': 'http://www.youtube.com/channel/UCQJLqOsy7dx7cMVHA-vozfA',
    'authorChannelId': {'value': 'UCQJLqOsy7dx7cMVHA-vozfA'},
    'textDisplay': 'They weren’t hooray',
    'textOriginal': 'They weren’t hooray',
    'parentId': 'UgwQTQvknQyoCTRAkUp4AaABAg',
    'canRate': True,
    'viewerRating': 'none',
    'likeCount': 0,
    'publishedAt': '2018-11-07T10:50:15.000Z',
    'updatedAt': '2018-11-07T10:50:15.000Z'}}]}

In [ ]:
x =  { "video_id": video_id, "video_url":my_url,\
      "video_title":video_title, "channel":channel,\
     "series":series, "video_snippet":description,\
     "video":video,"comment":comment,"reply":reply,\
     "video_op":video_op, "comment_op":comment_op,\
     "reply_op":reply_op,"date_posted":date_posted,\
     "content":transcript,"date_scrape":date_scrape}